# Challenge

There are three tables: 

## 1. Cards table : 


| id | brand            | type   |

+----+------------------+--------+

|  1 | Mastercard       | Credit |

|  2 | American Express | Credit |




## 2. Payees table: 



| id | name   | vertical           |

+----+--------+--------------------+

|  1 | A Corp | Rent & Real Estate |

|  2 | B Corp | Rent & Real Estate |

|  3 | C Corp | Rent & Real Estate |


## 3. Payments table:


| id | submitted_at        | success | payee_id | card_id | principal | fee    |

+----+---------------------+---------+----------+---------+-----------+--------+

|  1 | 2018-08-31 11:56:59 |       1 |        2 |      20 |  10960.90 |  14.01 |

|  2 | 2018-07-28 06:44:37 |       1 |        7 |     149 |  14170.00 | 354.25 |

|  3 | 2018-06-06 15:50:02 |       1 |       19 |      39 |  19365.00 | 484.13 |



Questions: 
1. Show all of the payees that have had at least $100,000 in successful payment volume (volume defined as principal + fee)  submitted in 2018. Please have the query output the payee id and associated 2018 successful volume figure.

2. A list of unsuccessful Visa payments submitted in the last 180 days rolling. Please have the query output the payment id, the name of the payee, the vertical of the payee and the type of card, as well as a flag column that returns 1 if the payment was greater than or equal to $10,000.00. (NOTE: for MySQL, curdate() will return the current date). 

3. A list of payees and the number of successful payments they have had. Please have the query output the payee id, the name of the payee, and the number of successful payments (if the payee has no successful payments, please have the payee included in the row, but the number or successful payments listed as 0). 

Answers: 

1. a) select payee_id, sum from (select payee_id, sum(principal+fee) as sum from payments where success =1 and year(submitted_at)=2018 group by payee_id) as table1 where sum>100000;
   b) select payee_id, sum(principal+fee) as payment_volume from payments where success = '1' and year(submitted_at)='2018' group by payee_id having payment_volume>100000; 
   c) select payee_id, sum(principal+fee) as total from (select * from payments where success=1 and year(submitted_at)=2018) as temptable group by payee_id having sum(principal+fee)>100000; 


| payee_id | sum       |

+----------+-----------+

|        2 | 152458.05 |

|        4 | 107251.32 |

|        6 | 122919.73 |

|        7 | 267560.46 |

|       11 | 444816.21 |

|       18 | 108819.45 |

|       19 | 100889.36 |


2.  select t1.id, t1.payee_id, t1.card_id, t2.name, t2.vertical, t3.type, t1.payment, case when t1.payment>=10000 then 1 when t1.payment<10000 then 0 end as flag_column from (select id, payee_id, card_id, principal+fee as payment  from payments where success = 0 and (curdate()-date(submitted_at))>180 ) as t1 join payees t2 on t2.id = t1.payee_id join cards t3 on t1.card_id=t3.id;



| id  | payee_id | card_id | name   | vertical           | type   | payment  | flag_column |

+-----+----------+---------+--------+--------------------+--------+----------+-------------+

|   7 |       11 |      75 | K Corp | Business Services  | Credit | 19257.70 |           1 |

|  15 |        7 |     124 | G Corp | Rent & Real Estate | Debit  | 22611.50 |           1 |

|  42 |        2 |     140 | B Corp | Rent & Real Estate | Credit | 12703.54 |           1 |

3. select t1.payee_id, t1.successful_payments, t2.name from (select payee_id, sum(success) as successful_payments from payments group by payee_id) as t1 join payees t2 on t1.payee_id = t2.id order by successful_payments desc;



| payee_id | successful_payments | name   |

+----------+---------------------+--------+

|       11 |                  38 | K Corp |

|        7 |                  25 | G Corp |

|        2 |                  17 | B Corp |

|        1 |                   9 | A Corp |

|        6 |                   9 | F Corp |

|       19 |                   8 | S Corp |

|        4 |                   8 | D Corp |

